<a href="https://colab.research.google.com/github/lciraci/AI_Projects/blob/main/_English_toItalian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install fsspec==2023.4.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requir

# parallel sentences: English → Italian.

### Download and preview the opus_books English-Italian translation dataset
### This script loads a public parallel corpus (EN-IT) from the Hugging Face Hub.
### After running the cell below, you'll see the available splits (like train/test)
### and the number of examples in each. Use this as a starting point for any translation/NLP project!


In [5]:
from datasets import load_dataset
dataset = load_dataset("opus_books", "en-it")
print(dataset)


HTTP Error 429 thrown while requesting GET https://huggingface.co/api/datasets/Helsinki-NLP/opus_books/tree/1f9f6191d0e91a3c539c2595e2fe48fc1420de9b?recursive=True&expand=True&cursor=ZXlKbWFXeGxYMjVoYldVaU9pSm1jaTF1YnlJc0luUnlaV1ZmYjJsa0lqb2lZakkxTm1FNE5HRmhORFpsWkRFeE9UZGtPRGMwT0dJMU56TmtOakF6T1RVeE9XRmlaRGM0TkNKOTo1MA%3D%3D
Retrying in 1s [Retry 1/20].


Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 32332
    })
})


### Show the first translation pair (EN → IT) in the training set

In [6]:
print(dataset['train'][0])

{'id': '0', 'translation': {'en': 'Source: Project Gutenberg', 'it': 'Source: www.liberliber.it/Audiobook available here'}}


### Update the first translation pair in the training set

In [7]:
from datasets import load_dataset, Dataset

# 1. Load
train_data = dataset['train']

# 2. Update the first sample
train_data_list = list(train_data)
train_data_list[0] = {
    'translation': {
        'en': "I'm italian.",
        'it': "Sono italiano."
    }
}

updated_train_data = Dataset.from_list(train_data_list)

# 3. Now you can use updated_train_data for training!
print(updated_train_data[0])


{'translation': {'en': "I'm italian.", 'it': 'Sono italiano.'}}


In [8]:
# Select first 1000 samples for quick testing
small_train = dataset['train'].select(range(1000))
print(small_train)
print(dataset['train'].column_names)


#explore more option on dataset


Dataset({
    features: ['id', 'translation'],
    num_rows: 1000
})
['id', 'translation']


In [9]:
# Export the train split to a CSV file
dataset['train'].to_csv("opus_books_train.csv")
from google.colab import files
files.download("opus_books_train.csv")


Creating CSV from Arrow format:   0%|          | 0/33 [00:00<?, ?ba/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Show a specific sample and update the first translation pair in the training


In [10]:
print(dataset['train'][190])
#Update
train_data_list = list(train_data)
train_data_list[0] = {
    'translation': {
        'en': "Well, who am I?"" he asked",
        'it': "Chi sono? Ha chiesto?"
    }
}
updated_train_data = Dataset.from_list(train_data_list)


{'id': '190', 'translation': {'en': '"Well, who am I?" he asked.', 'it': '— Chi sono? — domandò egli.'}}


### Remove the first 5 samples from the training set


In [11]:
# Remove the first 5 samples (indices 0 to 4)
train_data_list = train_data_list[5:]

# Re-create the Dataset
updated_train_data = Dataset.from_list(train_data_list)

# Tokenization

In [12]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-it"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/789k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/814k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-it', vocab_size=80035, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	80034: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [13]:
print(tokenizer)
print("Vocabulary size:", tokenizer.vocab_size)
print("Special tokens:", tokenizer.special_tokens_map)
print("First 10 tokens:", list(tokenizer.get_vocab().items())[:50])


MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-it', vocab_size=80035, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	80034: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
Vocabulary size: 80035
Special tokens: {'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}
First 10 tokens: [('</s>', 0), ('<unk>', 1), ('.', 2), (',', 3), ('▁the', 4), ("'", 5), ('▁di', 6), ('▁in', 7), ('▁of', 8), ('▁a', 9), ('▁e', 10), ('▁to', 11), ('s', 12), ('▁and', 13), ('▁-', 14), ('▁la', 15), ('▁', 16), ('▁che', 17), ('▁il', 18), ('?', 19), ('▁per', 20), (

In [14]:
sample_text = "My name is Lucio"
token_ids = tokenizer.encode(sample_text)
print("Token IDs:", token_ids)
print("Decoded back:", tokenizer.decode(token_ids))


Token IDs: [888, 842, 24, 6263, 564, 0]
Decoded back: My name is Lucio</s>


### For translation, you tokenize source (en) and target (it).
### When fine-tuning, you’ll want to create input_ids and labels for each sample.

### Tokenize the updated training data for model training
### This cell converts each translation pair into input IDs and attention masks
### for the model, and also prepares target labels for training.
### The 'attention_mask' ensures the model ignores padding tokens during learning.


In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-it")

def preprocess_function(batch):
    inputs = [example["en"] for example in batch["translation"]]
    targets = [example["it"] for example in batch["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = updated_train_data.map(preprocess_function, batched=True)


Map:   0%|          | 0/32327 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [16]:
print(tokenized_train[0])

{'id': '5', 'translation': {'en': 'We had been wandering, indeed, in the leafless shrubbery an hour in the morning; but since dinner (Mrs. Reed, when there was no company, dined early) the cold winter wind had brought with it clouds so sombre, and a rain so penetrating, that further out-door exercise was now out of the question.', 'it': "La mattina avevamo errato per un'ora nel boschetto spogliato di foglie, ma dopo pranzo (quando non vi erano invitati, la signora Reed desinava presto), il vento gelato d'inverno aveva portato seco nubi così scure e una pioggia così penetrante, che non si poteva pensare a nessuna escursione."}, 'input_ids': [148, 287, 173, 12353, 2955, 68, 3, 5829, 3, 7, 4, 20875, 1410, 1424, 227, 8828, 3968, 95, 3697, 7, 4, 2601, 63, 139, 944, 5217, 25, 28076, 12, 2, 304, 102, 3, 269, 181, 82, 138, 1196, 3, 7305, 102, 2848, 38, 4, 5086, 6208, 5471, 287, 2762, 41, 53, 10445, 12, 121, 121, 98, 3705, 3, 13, 9, 8122, 121, 41162, 3, 33, 1268, 150, 28, 9768, 5435, 82, 309, 1

# Choose and Load a Pre-trained Translation Model

---



In [17]:
from transformers import AutoModelForSeq2SeqLM

# Load the pre-trained MarianMT model for EN→IT translation
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-it")


### Define training arguments for fine-tuning the translation model
### This configures the output directory, batch size, number of epochs,
### learning rate, logging, checkpoint saving, and precision settings.


In [18]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    fp16=True,  # Set to False if no GPU
)


### Create a data collator to handle dynamic padding and batching for seq2seq training


In [19]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


### Set up and run the Hugging Face Trainer to fine-tune the translation model


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


<ipython-input-20-9a00a97565aa>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter: